In [ ]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
import ast
from time import gmtime, strftime
import re
import time
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

---
csv 파일 불러오기

In [ ]:
fnl_df_glowpick_prd_info = pd.read_csv("fnl_df_glowpick_prd_info_w_review.csv")
fnl_df_glowpick_prd_info = fnl_df_glowpick_prd_info.iloc[:, 1:]

In [ ]:
fnl_df_brd_w_prd = pd.read_csv("fnl_df_brd_w_prd.csv")
fnl_df_brd_w_prd = fnl_df_brd_w_prd.iloc[:, 1:]

for i in fnl_df_brd_w_prd.index:
    fnl_df_brd_w_prd.at[i, 'product_name'] = eval(fnl_df_brd_w_prd.loc[i, 'product_name'])
    fnl_df_brd_w_prd.at[i, 'product_link'] = eval(fnl_df_brd_w_prd.loc[i, 'product_link'])

In [ ]:
df_glowpick_prd = pd.read_csv("df_glowpick_prd.csv")
df_glowpick_prd = df_glowpick_prd.iloc[:, 1:]

df_glowpick_prd.columns = ['브랜드명_fix', '제품 id', '제품명_fix'] #구분을 위해 컬럼명 변경

---
DB 업로드를 위한 정렬, nan값 체크

In [ ]:
#브랜드 id와 브랜드 link값 채우기

for idx in tqdm(list(fnl_df_glowpick_prd_info.index)[:30000]):
    brd_name = fnl_df_glowpick_prd_info.loc[idx, '브랜드명']
    
    try:
        fnl_df_glowpick_prd_info.loc[idx, '브랜드 link'] = fnl_df_brd_w_prd.loc[fnl_df_brd_w_prd['brand_name'] == brd_name, 'brand_code']
    
    #브랜드명이 중복인 경우
    except ValueError:
        for i in list(fnl_df_brd_w_prd.index):
            if fnl_df_glowpick_prd_info.loc[idx, '제품 link'] in fnl_df_brd_w_prd.loc[i, 'product_link']:
                fnl_df_glowpick_prd_info.loc[idx, '브랜드 link'] = fnl_df_brd_w_prd.loc[i, 'brand_code']

In [ ]:
#bridge table의 제품 정렬 순서로 sorting
df_glowpick_prd_sort = pd.merge(df_glowpick_prd, fnl_df_glowpick_prd_info, how='inner', on='제품 id')

#브릿지 테이블에서 픽스된 정확한 브랜드명과 제품명을 사용하기 위해 재매핑
df_glowpick_prd_sort = df_glowpick_prd_sort.drop(['브랜드명'], axis = 'columns') #크롤링한 브랜드명 삭제
df_glowpick_prd_sort = df_glowpick_prd_sort.drop(['제품명'], axis = 'columns') #크롤링한 제품명 삭제

df_glowpick_prd_sort = df_glowpick_prd_sort.rename({'브랜드명_fix': '브랜드명'}, axis = 'columns') #브릿지 테이블에서 픽스된 브랜드명 사용
df_glowpick_prd_sort = df_glowpick_prd_sort.rename({'제품명_fix': '제품명'}, axis = 'columns') #브릿지 테이블에서 픽스된 제품명 사용

In [ ]:
df_glowpick_prd_sort = df_glowpick_prd_sort[columns] #컬럼 순서 변경

In [ ]:
#db에 올릴 테이블은 이미지 src 삭제
df_glowpick_prd_sort_drop_img_src = df_glowpick_prd_sort.drop('img_src')
df_glowpick_prd_sort_drop_img_src = df_glowpick_prd_sort_drop_img_src.reset_index(drop=True)
df_glowpick_prd_sort_drop_img_src.to_csv("fnl_df_glowpick_prd_info.csv")

df_glowpick_prd_sort = df_glowpick_prd_sort.reset_index(drop=True)
df_glowpick_prd_sort.to_csv("fnl_df_glowpick_prd_info_w_img_src.csv")

---

In [ ]:
glowpick_product_info_final_version_w_review = pd.read_csv("glowpick_product_info_final_version_w_review.csv")

In [ ]:
for i in tqdm(glowpick_product_info_final_version_w_review.index):
    try:
        if type(glowpick_product_info_final_version_w_review.loc[i, 'product_rating']) != float:
            glowpick_product_info_final_version_w_review.at[i, 'product_rating'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'product_rating'])
            glowpick_product_info_final_version_w_review.at[i, 'product_review_date'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'product_review_date'])
            glowpick_product_info_final_version_w_review.at[i, 'count_product_review'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'count_product_review'])
            glowpick_product_info_final_version_w_review.at[i, 'user_skin_type'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'user_skin_type'])
            glowpick_product_info_final_version_w_review.at[i, 'user_age'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'user_age'])
            glowpick_product_info_final_version_w_review.at[i, 'user_sex'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'user_sex'])
            glowpick_product_info_final_version_w_review.at[i, 'user_name'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'user_name'])
            glowpick_product_info_final_version_w_review.at[i, 'product_review'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'product_review'])
    except ValueError:
            pass

In [ ]:
#review 관련 columns의 length가 모두 같은지 확인(nan, 프로필 미등록 회원 ect.)

err = []

for i in glowpick_product_info_final_version_w_review.index:
    if type(glowpick_product_info_final_version_w_review.loc[i, 'product_rating']) != float:
        if len(glowpick_product_info_final_version_w_review.loc[i, 'product_rating']) != len(glowpick_product_info_final_version_w_review.loc[i, 'user_name']):
            err.append(i)
print(len(err))

In [ ]:
check = err[0]

print('rating: ', len(glowpick_product_info_final_version_w_review.loc[check, 'product_rating']))
print('user_skin_type: ', len(glowpick_product_info_final_version_w_review.loc[check, 'user_skin_type']))
print('age: ', len(glowpick_product_info_final_version_w_review.loc[check, 'user_age']))
print('sex: ', len(glowpick_product_info_final_version_w_review.loc[check, 'user_sex']))
print('name: ', len(glowpick_product_info_final_version_w_review.loc[check, 'user_name']))
print('review: ', len(glowpick_product_info_final_version_w_review.loc[check, 'product_review']))

In [ ]:
#리스트로 변경되지 않은 row: nan값을 None값으로
for i in err:
    try:
        glowpick_product_info_final_version_w_review.at[i, 'user_skin_type'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'user_skin_type'].replace('nan', 'None'))
        glowpick_product_info_final_version_w_review.at[i, 'user_age'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'user_age'].replace('nan', 'None'))
        glowpick_product_info_final_version_w_review.at[i, 'user_name'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'user_name'].replace('nan', 'None'))
        glowpick_product_info_final_version_w_review.at[i, 'user_sex'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'user_sex'].replace('nan', 'None'))
        glowpick_product_info_final_version_w_review.at[i, 'product_review'] = ast.literal_eval(glowpick_product_info_final_version_w_review.loc[i, 'product_review'].replace('nan', 'None'))
    except:
        pass

In [ ]:
#리스트로 타입 변환 후, None값을 다시 nan값으로
glowpick_product_info_final_version_w_review.loc[check, 'product_review'][glowpick_product_info_final_version_w_review.loc[check, 'product_review'].index(None)] = np.nan